In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader.data import DataReader
from datetime import datetime
import talib
import talib.abstract as tabs

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
tickets = first_table['Symbol'].values.tolist()
tickets.remove('BRK.B')
tickets.remove('BF.B')

In [3]:
all_stocks = []
for ticket in tickets:
    stocks = pd.read_csv('../datasets/yahoo/' + ticket + '.csv')
    stocks.rename(columns=str.lower, inplace=True)
    all_stocks.append(stocks)

In [5]:
def feature_extraction(df):
    
    # ROC 10
    roc = tabs.ROC(df, timeperiod=10)
    roc = np.nan_to_num(roc)
    df['roc'] = roc
    
    # SMA 10
    sma = tabs.SMA(df, timeperiod=10)
    sma = np.nan_to_num(sma)
    df['sma'] = sma
    
    # MACD, MACD SIGNAL and MACD HIST
    
    macd, macdsignal, macdhist = talib.MACD(df['close'])
    macd = np.nan_to_num(macd)
    macdsignal = np.nan_to_num(macdsignal)
    macdhist = np.nan_to_num(macdhist)
    df['macd'] = macd
    df['macd_signal'] = macdsignal
    df['macd_hist'] = macdhist
    
    # CCI 10
    cci = tabs.CCI(df, timeperiod=10)
    cci = np.nan_to_num(cci)
    df['cci'] = cci
    
    #     MTM 10 
    mtm = tabs.MOM(df, timeperiod=10)
    mtm = np.nan_to_num(mtm)
    df['mtm'] = mtm
    
    #     RSI 5 
    rsi = tabs.RSI(df, timeperiod=5)
    rsi = np.nan_to_num(rsi)
    df['rsi'] = rsi
    
    #     WNR 9
    wnr = tabs.WMA(df, timeperiod=9)
    wnr = np.nan_to_num(wnr)
    df['wnr'] = wnr
    
    #     SLOWK & SLOWD
    slowk, slowd = talib.STOCH(df['high'], df['low'], df['close'])
    slowk = np.nan_to_num(slowk)
    slowd = np.nan_to_num(slowd)
    df['slowk'] = slowk
    df['slowd'] = slowd
    
    #     ADOSC 
    adosc = tabs.ADOSC(df)
    adosc = np.nan_to_num(adosc)
    df['adosc'] = adosc
    
    #     AARON
    aroondown, aroonup = talib.AROON(df['high'], df['low'])
    aroondown = np.nan_to_num(aroondown)
    aroonup = np.nan_to_num(aroonup)
    df['aroon_down'] = aroondown
    df['aroon_up'] = aroonup
    
    #     BBANDS
    upper, middle, lower = talib.BBANDS(df['close'], matype=0)
    upper = np.nan_to_num(upper)
    df['upper'] = upper
    middle = np.nan_to_num(middle)
    df['middle'] = middle
    lower = np.nan_to_num(lower)
    df['bbands'] = lower
    
    # SAR
    sar = tabs.SAR(df)
    sar = np.nan_to_num(sar)
    df['sar'] = sar
    
    # BOP
    bop = tabs.BOP(df)
    bop = np.nan_to_num(bop)
    df['bop'] = bop
    
    # DX
    dx = tabs.DX(df)
    dx = np.nan_to_num(dx)
    df['dx'] = dx
    
    # PLUS_DI
    plus_di = tabs.PLUS_DI(df)
    plus_di = np.nan_to_num(plus_di)
    df['plus_di'] = plus_di
    
    # WIILR
    willr = tabs.WILLR(df)
    willr = np.nan_to_num(willr)
    df['willr'] = willr
    
    # TRIX
    trix = tabs.TRIX(df)
    trix = np.nan_to_num(trix)
    df['trix'] = trix
    
    # OBV
    obv = tabs.OBV(df)
    obv = np.nan_to_num(obv)
    df['obv'] = obv


In [7]:
for ticket, stock in zip(tickets, all_stocks):
    s = stock.copy()
    feature_extraction(s)
    s = s.iloc[10:]
    s.to_csv('../datasets/enriched/' + ticket + '.csv', index=False)

In [197]:
all_stocks = []
for ticket in tickets:
    stocks = pd.read_csv('../datasets/enriched/' + ticket + '.csv')
    stocks = stocks[stocks.columns[:-7]]
    all_stocks.append(stocks)

In [229]:
dataset = pd.concat(all_stocks, axis=0)
dataset

,high,low,open,close,volume,adj close,roc,sma,macd,macd_signal,...,rsi,wnr,slowk,slowd,adosc,aroon_down,aroon_up,upper,middle,bbands
0,206.789993,202.190002,202.869995,206.570007,2623000.0,191.088211,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
1,210.399994,206.589996,207.089996,209.720001,2967300.0,194.002090,1.524904,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2,207.119995,204.050003,206.589996,204.929993,2229700.0,190.881744,-2.284002,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
3,208.970001,206.000000,206.460007,208.860001,2000400.0,194.542313,1.917732,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4,209.149994,207.389999,207.910004,208.410004,1534400.0,194.123199,-0.215454,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,211.148209,207.698001,204.247793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,161.889999,158.500000,160.399994,160.020004,1625100.0,160.020004,0.464594,156.8270,-1.440415,-1.428655,...,62.246209,157.420890,74.490395,63.224442,-8.872658e+05,57.142857,14.285714,161.197062,157.810001,154.422939
502,161.789993,159.009995,159.809998,159.830002,1852700.0,159.830002,-0.118737,156.8390,-1.227758,-1.388476,...,60.704453,158.085557,76.644004,70.838388,-9.460893e+05,50.000000,7.142857,161.991619,158.298001,154.604384
503,162.729996,160.169998,161.250000,160.619995,1746000.0,160.619995,0.494271,157.5860,-0.984136,-1.307608,...,65.186027,158.759778,73.449505,74.861301,-1.244383e+06,42.857143,0.000000,161.779680,159.376001,156.972322
504,164.729996,160.399994,161.149994,164.440002,1860300.0,164.440002,2.378289,158.3770,-0.477318,-1.141550,...,79.392055,160.107112,80.620873,76.904794,-7.406573e+05,35.714286,0.000000,164.540377,160.838000,157.135624


In [230]:
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [231]:
NUM_FEATURES = 20

In [232]:
from sklearn.decomposition import PCA
pca = PCA(n_components = NUM_FEATURES)
pca.fit(dataset)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [233]:
pca.explained_variance_ratio_

array([5.65869866e-01, 1.83491944e-01, 1.46459219e-01, 5.93871958e-02,
       2.88988466e-02, 8.79215094e-03, 2.95258946e-03, 1.55300439e-03,
       1.14434016e-03, 5.73754605e-04, 4.82080705e-04, 2.11204435e-04,
       1.14396612e-04, 3.94793508e-05, 1.33801178e-05, 9.03816269e-06,
       4.79224965e-06, 1.61874399e-06, 5.57469089e-07, 3.85740536e-07])

In [234]:
def prepare_input_data(X, timerange=10, split=0.1):
    
    train_size = int(X.shape[0] * (1 - split))
    train_size = min(train_size, X.shape[0] - timerange)
    
    test_size = X.shape[0] - train_size - timerange
    
    X_train = np.ndarray(shape=(train_size, timerange, X.shape[1]))
    for i in range(train_size):
        X_train[i] = X[i:i+timerange]
    
    X_test = np.ndarray(shape=(test_size, timerange, X.shape[1]))
    for i in range(test_size):
        X_test[i] = X[train_size+i:train_size+i+timerange]
        
    return X_train, X_test
    
def prepare_output_data(X, timerange=10, split=0.1):
    
    train_size = int(X.shape[0] * (1 - split))
    train_size = min(train_size, X.shape[0] - timerange)
    
    test_size = X.shape[0] - train_size - timerange
    
    Y_train = np.ndarray(shape=(train_size))
    for i in range(train_size):
        Y_train[i] = 1 if X[i+timerange-1] < X[i+timerange] else 0
    
    Y_test = np.ndarray(shape=(test_size))
    for i in range(test_size):
        Y_test[i] = 1 if X[train_size+i+timerange-1] < X[train_size+i+timerange] else 0
    
    return Y_train, Y_test

In [235]:
TIMERANGE = 10

X_train, X_test = [np.ndarray(shape=(0, TIMERANGE, NUM_FEATURES)) for i in range(2)]
Y_train, Y_test = [np.ndarray(shape=(0)) for i in range(2)]
for stock in all_stocks:
    X = scaler.transform(stock)
    X = pca.transform(X)
    x_train, x_test = prepare_input_data(X, timerange=TIMERANGE)
    y_train, y_test = prepare_output_data(stock['close'], timerange=TIMERANGE)
    X_train = np.append(X_train, x_train, axis=0)
    Y_train = np.append(Y_train, y_train, axis=0)
    X_test = np.append(X_test, x_test, axis=0)
    Y_test = np.append(Y_test, y_test, axis=0)

In [236]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((227410, 10, 20), (20462, 10, 20), (227410,), (20462,))

In [246]:
# from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

# create and compile the LSTM network
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer=Adam(0.001), metrics=['accuracy'])

In [247]:
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=50, batch_size=256)

Epoch 1/50
800/800 [==============================] - 37s 46ms/step - loss: 0.2690 - accuracy: 0.5064 - val_loss: 0.2493 - val_accuracy: 0.5216
Epoch 2/50
800/800 [==============================] - 35s 43ms/step - loss: 0.2512 - accuracy: 0.5135 - val_loss: 0.2493 - val_accuracy: 0.5263
Epoch 3/50
800/800 [==============================] - 42s 52ms/step - loss: 0.2510 - accuracy: 0.5135 - val_loss: 0.2491 - val_accuracy: 0.5270
Epoch 4/50
800/800 [==============================] - 37s 46ms/step - loss: 0.2506 - accuracy: 0.5159 - val_loss: 0.2498 - val_accuracy: 0.5034
Epoch 5/50
800/800 [==============================] - 34s 42ms/step - loss: 0.2506 - accuracy: 0.5161 - val_loss: 0.2490 - val_accuracy: 0.5286
Epoch 6/50
800/800 [==============================] - 46s 57ms/step - loss: 0.2504 - accuracy: 0.5184 - val_loss: 0.2489 - val_accuracy: 0.5286
Epoch 7/50
800/800 [==============================] - 32s 40ms/step - loss: 0.2500 - accuracy: 0.5198 - val_loss: 0.2491 - val_accuracy:

In [249]:
model.evaluate(X_test, Y_test)

640/640 [==============================] - 2s 3ms/step - loss: 0.2491 - accuracy: 0.5210


[0.24910366535186768, 0.5209656953811646]

In [254]:
# from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

histories = []
NUM_EPOCHS=50
BATCH_SIZE=256

for TIMERANGE in range(2, 15):
    
    X_train, X_test = [np.ndarray(shape=(0, TIMERANGE, NUM_FEATURES)) for i in range(2)]
    Y_train, Y_test = [np.ndarray(shape=(0)) for i in range(2)]
    for stock in all_stocks:
        X = scaler.transform(stock)
        X = pca.transform(X)
        x_train, x_test = prepare_input_data(X, timerange=TIMERANGE)
        y_train, y_test = prepare_output_data(stock['close'], timerange=TIMERANGE)
        X_train = np.append(X_train, x_train, axis=0)
        Y_train = np.append(Y_train, y_train, axis=0)
        X_test = np.append(X_test, x_test, axis=0)
        Y_test = np.append(Y_test, y_test, axis=0)

    model = Sequential()
    
    model.add(LSTM(units = 50, return_sequences = True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))

    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam(0.001), metrics=['accuracy'])
    
    
    history = model.fit(X_train, Y_train, validation_split=0.1, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
    
    histories.append(history)
    
    model.save('../models/lstm-timerage-' + str(TIMERANGE))
    
    print('model evaluation: ', model.evaluate(X_test, Y_test))
    

Epoch 1/50
800/800 [==============================] - 10s 12ms/step - loss: 0.2575 - accuracy: 0.5166 - val_loss: 0.2501 - val_accuracy: 0.5025
Epoch 2/50
800/800 [==============================] - 4s 4ms/step - loss: 0.2501 - accuracy: 0.5206 - val_loss: 0.2486 - val_accuracy: 0.5337
Epoch 3/50
800/800 [==============================] - 7s 8ms/step - loss: 0.2494 - accuracy: 0.5254 - val_loss: 0.2488 - val_accuracy: 0.5316
Epoch 4/50
800/800 [==============================] - 5s 7ms/step - loss: 0.2489 - accuracy: 0.5297 - val_loss: 0.2480 - val_accuracy: 0.5403
Epoch 5/50
800/800 [==============================] - 5s 7ms/step - loss: 0.2484 - accuracy: 0.5344 - val_loss: 0.2475 - val_accuracy: 0.5368
Epoch 6/50
800/800 [==============================] - 5s 7ms/step - loss: 0.2481 - accuracy: 0.5352 - val_loss: 0.2475 - val_accuracy: 0.5359
Epoch 7/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2480 - accuracy: 0.5353 - val_loss: 0.2473 - val_accuracy: 0.5385
Epoc

Epoch 7/50
800/800 [==============================] - 6s 7ms/step - loss: 0.2477 - accuracy: 0.5391 - val_loss: 0.2468 - val_accuracy: 0.5431
Epoch 8/50
800/800 [==============================] - 6s 7ms/step - loss: 0.2474 - accuracy: 0.5398 - val_loss: 0.2466 - val_accuracy: 0.5448
Epoch 9/50
800/800 [==============================] - 6s 8ms/step - loss: 0.2471 - accuracy: 0.5419 - val_loss: 0.2465 - val_accuracy: 0.5456
Epoch 10/50
800/800 [==============================] - 6s 7ms/step - loss: 0.2469 - accuracy: 0.5422 - val_loss: 0.2463 - val_accuracy: 0.5448
Epoch 11/50
800/800 [==============================] - 8s 9ms/step - loss: 0.2466 - accuracy: 0.5437 - val_loss: 0.2462 - val_accuracy: 0.5473
Epoch 12/50
800/800 [==============================] - 7s 8ms/step - loss: 0.2464 - accuracy: 0.5443 - val_loss: 0.2459 - val_accuracy: 0.5460
Epoch 13/50
800/800 [==============================] - 8s 10ms/step - loss: 0.2460 - accuracy: 0.5456 - val_loss: 0.2454 - val_accuracy: 0.5476
E

Epoch 13/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2447 - accuracy: 0.5513 - val_loss: 0.2442 - val_accuracy: 0.5551
Epoch 14/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2443 - accuracy: 0.5534 - val_loss: 0.2437 - val_accuracy: 0.5550
Epoch 15/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2438 - accuracy: 0.5543 - val_loss: 0.2432 - val_accuracy: 0.5563
Epoch 16/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2434 - accuracy: 0.5558 - val_loss: 0.2429 - val_accuracy: 0.5569
Epoch 17/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2431 - accuracy: 0.5568 - val_loss: 0.2426 - val_accuracy: 0.5590
Epoch 18/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2427 - accuracy: 0.5579 - val_loss: 0.2423 - val_accuracy: 0.5613
Epoch 19/50
800/800 [==============================] - 7s 9ms/step - loss: 0.2425 - accuracy: 0.5594 - val_loss: 0.2420 - val_accuracy: 0.5602

Epoch 19/50
800/800 [==============================] - 9s 11ms/step - loss: 0.2409 - accuracy: 0.5631 - val_loss: 0.2403 - val_accuracy: 0.5610
Epoch 20/50
800/800 [==============================] - 9s 11ms/step - loss: 0.2403 - accuracy: 0.5655 - val_loss: 0.2394 - val_accuracy: 0.5645
Epoch 21/50
800/800 [==============================] - 10s 12ms/step - loss: 0.2400 - accuracy: 0.5658 - val_loss: 0.2389 - val_accuracy: 0.5687
Epoch 22/50
800/800 [==============================] - 10s 12ms/step - loss: 0.2393 - accuracy: 0.5679 - val_loss: 0.2388 - val_accuracy: 0.5686
Epoch 23/50
800/800 [==============================] - 9s 11ms/step - loss: 0.2391 - accuracy: 0.5691 - val_loss: 0.2384 - val_accuracy: 0.5697
Epoch 24/50
800/800 [==============================] - 10s 12ms/step - loss: 0.2385 - accuracy: 0.5711 - val_loss: 0.2376 - val_accuracy: 0.5697
Epoch 25/50
800/800 [==============================] - 9s 12ms/step - loss: 0.2381 - accuracy: 0.5723 - val_loss: 0.2373 - val_accura

800/800 [==============================] - 11s 14ms/step - loss: 0.2345 - accuracy: 0.5819 - val_loss: 0.2346 - val_accuracy: 0.5814
Epoch 30/50
800/800 [==============================] - 12s 15ms/step - loss: 0.2341 - accuracy: 0.5833 - val_loss: 0.2342 - val_accuracy: 0.5804
Epoch 31/50
800/800 [==============================] - 18s 22ms/step - loss: 0.2335 - accuracy: 0.5839 - val_loss: 0.2340 - val_accuracy: 0.5846
Epoch 32/50
800/800 [==============================] - 14s 17ms/step - loss: 0.2330 - accuracy: 0.5855 - val_loss: 0.2337 - val_accuracy: 0.5821
Epoch 33/50
800/800 [==============================] - 13s 16ms/step - loss: 0.2327 - accuracy: 0.5871 - val_loss: 0.2324 - val_accuracy: 0.5865
Epoch 34/50
800/800 [==============================] - 19s 23ms/step - loss: 0.2321 - accuracy: 0.5884 - val_loss: 0.2323 - val_accuracy: 0.5888
Epoch 35/50
800/800 [==============================] - 21s 26ms/step - loss: 0.2317 - accuracy: 0.5897 - val_loss: 0.2322 - val_accuracy: 0.58

800/800 [==============================] - 14s 17ms/step - loss: 0.2268 - accuracy: 0.6022 - val_loss: 0.2282 - val_accuracy: 0.5990
Epoch 40/50
800/800 [==============================] - 14s 17ms/step - loss: 0.2263 - accuracy: 0.6040 - val_loss: 0.2283 - val_accuracy: 0.5971
Epoch 41/50
800/800 [==============================] - 13s 16ms/step - loss: 0.2261 - accuracy: 0.6047 - val_loss: 0.2280 - val_accuracy: 0.5999
Epoch 42/50
800/800 [==============================] - 13s 17ms/step - loss: 0.2256 - accuracy: 0.6061 - val_loss: 0.2275 - val_accuracy: 0.6017
Epoch 43/50
800/800 [==============================] - 13s 17ms/step - loss: 0.2251 - accuracy: 0.6070 - val_loss: 0.2271 - val_accuracy: 0.6019
Epoch 44/50
800/800 [==============================] - 14s 17ms/step - loss: 0.2248 - accuracy: 0.6078 - val_loss: 0.2275 - val_accuracy: 0.5999
Epoch 45/50
800/800 [==============================] - 20s 25ms/step - loss: 0.2244 - accuracy: 0.6097 - val_loss: 0.2265 - val_accuracy: 0.60

800/800 [==============================] - 15s 19ms/step - loss: 0.2201 - accuracy: 0.6190 - val_loss: 0.2250 - val_accuracy: 0.6063
Epoch 50/50
800/800 [==============================] - 15s 19ms/step - loss: 0.2201 - accuracy: 0.6191 - val_loss: 0.2243 - val_accuracy: 0.6074
INFO:tensorflow:Assets written to: ../models/lstm-timerage-11/assets
624/624 [==============================] - 1s 2ms/step - loss: 0.2585 - accuracy: 0.5090
model evaluation:  [0.25848427414894104, 0.508967936038971]
Epoch 1/50
800/800 [==============================] - 16s 20ms/step - loss: 0.2542 - accuracy: 0.5147 - val_loss: 0.2494 - val_accuracy: 0.5196
Epoch 2/50
800/800 [==============================] - 17s 21ms/step - loss: 0.2500 - accuracy: 0.5220 - val_loss: 0.2484 - val_accuracy: 0.5364
Epoch 3/50
800/800 [==============================] - 17s 21ms/step - loss: 0.2492 - accuracy: 0.5293 - val_loss: 0.2484 - val_accuracy: 0.5355
Epoch 4/50
800/800 [==============================] - 17s 21ms/step - lo

800/800 [==============================] - 20s 25ms/step - loss: 0.2465 - accuracy: 0.5415 - val_loss: 0.2460 - val_accuracy: 0.5472
Epoch 8/50
800/800 [==============================] - 20s 25ms/step - loss: 0.2459 - accuracy: 0.5441 - val_loss: 0.2452 - val_accuracy: 0.5498
Epoch 9/50
800/800 [==============================] - 21s 27ms/step - loss: 0.2453 - accuracy: 0.5464 - val_loss: 0.2445 - val_accuracy: 0.5505
Epoch 10/50
800/800 [==============================] - 21s 27ms/step - loss: 0.2448 - accuracy: 0.5480 - val_loss: 0.2438 - val_accuracy: 0.5542
Epoch 11/50
800/800 [==============================] - 21s 27ms/step - loss: 0.2441 - accuracy: 0.5507 - val_loss: 0.2431 - val_accuracy: 0.5565
Epoch 12/50
800/800 [==============================] - 27s 34ms/step - loss: 0.2432 - accuracy: 0.5537 - val_loss: 0.2421 - val_accuracy: 0.5596
Epoch 13/50
800/800 [==============================] - 20s 25ms/step - loss: 0.2422 - accuracy: 0.5576 - val_loss: 0.2417 - val_accuracy: 0.5597